# This script processes the file All_Optimizations.csv  

1. Generate the file `<hostname>/All_Optimizations-<hostname>.txt` by running:
    1. `process_baseline_results.ipynb` (for Baseline)
    2. `process_o1quadtree_results.ipynb` (for Opt1-Quadtree)
        1. `process_o1rev1_results.ipynb` (for Opt1-Rev1)
    3. `process_o2partree_results.ipynb` (for Opt2-parallization)
    4. `process_o3memo_results.ipynb` (for Opt3-memoization)
    5. `process_o4minrad_maxnum.ipynb` (for Opt4-minradius and maxnumber optimizations)

2. Run the next cells in this notebook...

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
from common.utils import get_best_optimization

# get the hostname of the server
hostname = os.popen("hostname").read().strip()

df=pd.read_csv(os.path.join(hostname, f'All_Optimizations-{hostname}.csv'),sep=';')
df.insert(4,"Total",0)
df['Total']=df['TimeTree']+df['TimeOWM']
df

In [ ]:
# get baseline
base = df.loc[df['Optimization'].str.contains('Baseline'), 'TimeTree':'Total'].copy()
print(base)
# get the best optimization
best,best_label = get_best_optimization(df)
print(best, best_label)

In [ ]:
b=np.array(base)
c=np.array(best)
speedup=b/c
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}x".format(x)})
#Print total speedup for each cloud (one cloud per row, one column for ech time measurement)
print(speedup)

# Speedup of each optimization w.r.t. the previous one

In [ ]:
df['TreeSp']=1.
df['OWMSp']=1.
df['TotalSp']=1.

#Speedup of one optimization w.r.t. the previous one
for i in range(4,len(df)):
    df.loc[i,'TreeSp']=df['TimeTree'][i-4]/df['TimeTree'][i]
    df.loc[i,'OWMSp']=df['TimeOWM'][i-4]/df['TimeOWM'][i]
    df.loc[i,'TotalSp']=df['Total'][i-4]/df['Total'][i]
df

# Important notice

**The sum of the relative speedups (one optim. wrt the previous one) is not the total speedup**

In [ ]:
df.groupby('Cloud').sum().drop(['TimeTree','TimeOWM','Total','Level','MinRadMaxNum'],axis=1)

# Speedup of each optimization w.r.t. baseline

In [ ]:
df=pd.read_csv(os.path.join(hostname, f'All_Optimizations-{hostname}.csv'),sep=';')
df.insert(4,"Total",0)
df['Total']=df['TimeTree']+df['TimeOWM']
df['TreeSp']=1.
df['OWMSp']=1.
df['TotalSp']=1.
#Speedup of one optimization w.r.t. baseline
for i in range(4,len(df),4):
    for j in range(4):
        df.loc[i+j,'TreeSp']=df['TimeTree'][j]/df['TimeTree'][i+j]
        df.loc[i+j,'OWMSp']=df['TimeOWM'][j]/df['TimeOWM'][i+j]
        df.loc[i+j,'TotalSp']=df['Total'][j]/df['Total'][i+j]
df

In [ ]:
df.loc[df['Cloud'].str.contains('Alcoy')]['OWMSp']

In [ ]:
df.loc[df['Cloud'].str.contains('Alcoy')]['OWMSp'] / df.loc[df['Cloud'].str.contains('Alcoy')]['OWMSp'].max()*100

# Improvement Factor for each cloud

In [ ]:
def plot_allsp(df,xlab):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    all=np.array(df.loc[4:,'TreeSp':'TotalSp'])
    clouds=list(df.loc[0:3,'Cloud'])
    bycloud={}
    for i,j in zip(clouds,range(len(clouds))):
        bycloud[i]=all[j::4].T

    #fig = plt.figure()
    labels=['OWM Trav.','Tree Const.','Total']
    x=np.arange(1,len(xlab)+1)
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True, sharey=True)
    for i,name in zip(range(len(clouds)),clouds):
        axs[i].plot(x, bycloud[name][1], marks[0], linewidth=linew, markersize=markers)
        axs[i].plot(x, bycloud[name][0], marks[1], linewidth=linew, markersize=markers)
        axs[i].plot(x, bycloud[name][2], marks[2], linewidth=linew, markersize=markers)

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Optimization', fontsize=xlabelfs)
        axs[i].set_xticks(x,labels=xlab,fontsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()
    fig.suptitle(f'Improvement factor of each optimization wrt baseline @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    
    axs[0].set_ylabel('Improvement Factor', fontsize=ylabelfs)
    axs[0].legend(labels,loc='best', fontsize= 14)
    pp = PdfPages(os.path.join(hostname, f"Speedup_all_optim-{hostname}.pdf"))
    pp.savefig(fig)
    pp.close()
    #axs[i].show()

# select only o1,o2,o3A,o3B,o3AB,o4R,o4N
dfsel = df.loc[df['Optimization'].isin(['Baseline','Opt1-Quadtree','Opt2-Par','Opt3-MemoA','Opt3-MemoB','Opt3-Memo','Opt4-MinRad','Opt4-MaxNum'])]

plot_allsp(dfsel,['1','2','3A','3B','3AB','R','N'])

In [ ]:
# drop REV1, memoA, memoB, and maxNumber
dfsel = df.loc[~df['Optimization'].str.contains('REV1|MemoA|MemoB|MaxNum')].copy()
display(dfsel)
plot_allsp(dfsel,['O1','O2','O3','O4'])

In [ ]:
# drop baseline and keep only the column Optimization and the columns from TreeSp to TotalSp
all_df = dfsel.loc[~dfsel['Optimization'].str.contains('Baseline')].copy()
# get the position of the best optimization in the dataframe
best_pos = all_df['Optimization'].unique().tolist().index(best_label)
print(f'{best_label} is at position {best_pos}')
# keep the speedups in an array
all_a = np.array(all_df.loc[:, 'TreeSp':'TotalSp'])
# get the clouds name
clouds = list(all_df.loc[all_df['Optimization'].str.contains(best_label),'Cloud'])
print(clouds)
bycloud={}
# create a dictionary with the speedups for each cloud
for i,cloud in enumerate(clouds):
    bycloud[cloud]=all_a[i::4]

# normalize the speedups to the best optimization
for i in clouds:
    # bycloud[i] = bycloud[i] / bycloud[i][best_pos]*100
    # the problem in this step is that best_pos points to the best optimization wrt Total time, but O2 or O3 could have a slightly better speedup wrt Tree construction an this would distort the next plot
    # reduce along the axis 0 performing max
    lmax = bycloud[i].max(axis=0)
    # check if the maximum is not in best_pos
    if not all(np.isclose(lmax, bycloud[i][best_pos])):
        print(f'Warning: the maximum for all cases for {i} is not in position {best_pos}')
        print(lmax)
        print(bycloud[i][best_pos])
    bycloud[i] = bycloud[i] / lmax*100
# print(bycloud)

# print("Adapt for stacked bar plot:")    
for i in clouds:
    for j in range(3,0,-1):
        bycloud[i][j]=bycloud[i][j]-bycloud[i][j-1]
    # print(bycloud[i])

# the order of the indices is: cloud, optimization, time
all_norm=np.array([bycloud[x] for x in clouds])
# print(f'Shape: {all_norm.shape}')
#see this to understand the transpose https://stackoverflow.com/questions/32034237/how-does-numpys-transpose-method-permute-the-axes-of-an-array
# now the order of the indices is: time, optimization, cloud
all_t=all_norm.transpose(2,1,0)
# print(all_t)

In [ ]:
def plot_allsp100(clouds, npa):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']


    #fig = plt.figure()
    plots=['Tree Const.','OWM Trav.','Total']
    # clouds=['Alcoy','Arzua','BrionF','BrionU']
    x=np.arange(1,len(clouds)+1)
    optim=['O1','O2','O3','O4']
    width=0.35
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 5), constrained_layout=True, sharey=True)
    for i,name in enumerate(plots):
        b=np.zeros(len(clouds))
        for z,j in enumerate(optim):
            axs[i].bar(x, npa[i][z], width, label=j, bottom=b)
            b=b+npa[i][z]

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Cloud', fontsize=xlabelfs)
        axs[i].set_xticks(x,labels=clouds,fontsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()
        
    fig.suptitle(f'Relative improvement factor of each optimization wrt baseline @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    
    axs[0].set_ylabel('% improvement factor', fontsize=ylabelfs)
    axs[0].legend(optim,loc='best', fontsize= 14)
    pp = PdfPages(os.path.join(hostname, f"Speedup_all_optim_ratio-{hostname}.pdf"))
    pp.savefig(fig)
    pp.close()

plot_allsp100(clouds, all_t)

# Build LaTeX table with speedups due to all optimizations (relative to baseline)

In [ ]:
# drop baseline and keep only the column Optimization and the columns from TreeSp to TotalSp
all_df = dfsel.loc[~dfsel['Optimization'].str.contains('Baseline')].copy()
# keep the speedups in an array
all_a = np.array(all_df.loc[:, 'TreeSp':'TotalSp'])
# get the clouds name
clouds = list(all_df.loc[all_df['Optimization'].str.contains(best_label),'Cloud'])

bycloud={}
# create a dictionary with the speedups for each cloud
for i,cloud in enumerate(clouds):
    bycloud[cloud]=all_a[i::4]

np.set_printoptions(formatter={'float': lambda x: "& {0:0.2f}x ".format(x)})    
bycloud

In [ ]:
print("\\begin{tabular}{|c|ccc|ccc|ccc|ccc|}\hline")
print("Cloud & \multicolumn{3}{c|}{O1} & \multicolumn{3}{c|}{O2} & \multicolumn{3}{c|}{O3} & \multicolumn{3}{c|}{O4} \\\\ \hline")  
print(" & Tree & OWM & Tot & Tree & OWM & Tot & Tree & OWM & Tot & Tree & OWM & Tot \\\\ \hline")
for i in clouds:
    print(i,end='')
    for j in range(4):
        for k in range(3):
            print("& {0:0.2f}x ".format(bycloud[i][j][k]),end='')

    print("\\\\ \hline")
print("\\end{tabular}")

# Build LaTeX table with speedups due to all optimizations (relative to the previous one)

In [ ]:
df=pd.read_csv(os.path.join(hostname, f'All_Optimizations-{hostname}.csv'), sep=';')
df.insert(4,"Total",0)
df['Total']=df['TimeTree']+df['TimeOWM']
df['TreeSp']=1.
df['OWMSp']=1.
df['TotalSp']=1.
# drop REV1, memoA, memoB, and maxNumber
dfsel = df.loc[~df['Optimization'].str.contains('REV1|MemoA|MemoB|MaxNum')].copy()
dfsel.reset_index(drop=True, inplace=True)

# Speedup of one optimization w.r.t. the previous one
for i in range(4,len(dfsel)):
    dfsel.loc[i,'TreeSp']=dfsel['TimeTree'][i-4]/dfsel['TimeTree'][i]
    dfsel.loc[i,'OWMSp']=dfsel['TimeOWM'][i-4]/dfsel['TimeOWM'][i]
    dfsel.loc[i,'TotalSp']=dfsel['Total'][i-4]/dfsel['Total'][i]
dfsel.reset_index(drop=True, inplace=True)

# drop baseline and keep only the column Optimization and the columns from TreeSp to TotalSp
all_df = dfsel.loc[~dfsel['Optimization'].str.contains('Baseline')].copy()
# keep the speedups in an array
all_a = np.array(all_df.loc[:, 'TreeSp':'TotalSp'])
# get the clouds name
clouds = list(all_df.loc[all_df['Optimization'].str.contains(best_label),'Cloud'])
bycloud={}
# create a dictionary with the speedups for each cloud
for i,cloud in enumerate(clouds):
    bycloud[cloud]=all_a[i::4]

np.set_printoptions(formatter={'float': lambda x: "& {0:0.2f}x ".format(x)})    
bycloud

In [ ]:
print("\\begin{tabular}{|c|ccc|ccc|ccc|ccc|}\hline")
print(" & \multicolumn{3}{c|}{O1 vs OpenMP baseline} & \multicolumn{3}{c|}{O2 vs O1} & \multicolumn{3}{c|}{O3 vs O2} &\multicolumn{3}{c|}{O4 vs O3}\\\\")  
print("Cloud   & Tree C. & OWM & Total & Tree C. & OWM & Total & Tree C. & OWM & Total &  Tree C. & OWM & Total \\\\ \hline")
for i in clouds:
    print(i,end='')
    for j in range(4):
        for k in range(3):
            print("& {0:0.2f}x ".format(bycloud[i][j][k]),end='')

    print("\\\\ \hline")
print("\\end{tabular}")

# Build LaTeX table with speedups due to all optimizations (relative to REV1)

In [ ]:
df=pd.read_csv(os.path.join(hostname, f'All_Optimizations-{hostname}.csv'), sep=';')
df.insert(4,"Total",0)
df['Total']=df['TimeTree']+df['TimeOWM']
df['TreeSp']=1.
df['OWMSp']=1.
df['TotalSp']=1.
# drop memoA, memoB, and maxNumber
dfsel = df.loc[~df['Optimization'].str.contains('MemoA|MemoB|MaxNum')].copy()
dfsel.reset_index(drop=True, inplace=True)

# get the position of Opt1-REV1
pos = dfsel.loc[dfsel['Optimization'].str.contains('Opt1-REV1')].index[0]

#Speedup of one optimization w.r.t. REV1 pos
for i in range(0,len(dfsel),4):
    for j in range(4):
        dfsel.loc[i+j,'TreeSp']=dfsel['TimeTree'][pos+j]/dfsel['TimeTree'][i+j]
        dfsel.loc[i+j,'OWMSp']=dfsel['TimeOWM'][pos+j]/dfsel['TimeOWM'][i+j]
        dfsel.loc[i+j,'TotalSp']=dfsel['Total'][pos+j]/dfsel['Total'][i+j]

# get only O2
dfo2 = dfsel.loc[dfsel['Optimization'].isin(['Opt2-Par'])].copy()
dfo2.reset_index(drop=True, inplace=True)

print(f"TBB is still {(dfo2['OWMSp'].mean()-1)*100:.1f}% faster than OMP with collapse and the tuned dynamic scheduler on average for the 4 clouds, {(dfo2['OWMSp'][0]-1)*100:.1f}%, {(dfo2['OWMSp'][1]-1)*100:.1f}%, {(dfo2['OWMSp'][2]-1)*100:.1f}%, {(dfo2['OWMSp'][3]-1)*100:.1f}% faster for Alcoy, Arzua, BrionF and BrionU, respectively.")